# 5 - GPT

Nosso objetivo é usar redes neurais para gerar algo que se pareça com português. Mais especificamente, algo que se pareça com um português específico --- aquele usado por Machado de Assis.

O código desse notebook é adaptado do repositório `nn-zero-to-hero`, de Andrej Karpathy. Os notebooks correspondentes (em inglês) podem ser encontrados na seguinte URL: https://github.com/karpathy/nn-zero-to-hero/blob/master/lectures/makemore/.

-------------------------------

Agora, munidos do nosso conhecimento de redes neurais, vamos testar uma arquitetura específica: GPT. Essa rede, por sua vez, é a base do ChatGPT.

Pra ser mais preciso, o GPT mistura duas coisas: (i) um encoder, capaz de entender uma noção de contexto (e.g., um usuário perguntando quem é o atual presidente do Brasil); e (ii) um decoder, capaz de gerar texto respeitando esse pedido (e.g., uma resposta coerente como "O atual presidente é ..."). Considerando nossa missão de gerar português baseado em Machado de Assis, vamos nos preocupar apenas com (ii) hoje.

A estrutura-base do GPT, um transformer, está especificada abaixo.

![](https://substack-post-media.s3.amazonaws.com/public/images/d9a0766d-2e52-4af0-96c5-3e07a30d6ecb_1868x1130.png)

Vamos implementar, em código, cada um dos passos do decoder.

## 5.1 Criando os inputs e outputs para a rede

In [1]:
import json
import os
import random

import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm

In [2]:
with open("../data/gpt/processed/machado-all.txt", "r") as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: entra string, saem inteiros
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: entram inteiros, sai uma string

In [3]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([ 7, 19, 18, 24, 19,  1,  0,  7, 19, 18, 24, 19, 23,  0, 10, 16, 25, 17,
        13, 18,  9, 18, 23,  9, 23,  1,  0,  7, 19, 18, 24, 19, 23,  0, 10, 16,
        25, 17, 13, 18,  9, 18, 23,  9, 23,  0, 24,  9, 28, 24, 19,  2, 10, 19,
        18, 24,  9,  0, 19,  6, 22,  5,  0,  7, 19, 17, 20, 16,  9, 24,  5,  1,
         0, 17,  5,  7, 12,  5,  8, 19,  0,  8,  9,  0,  5, 23, 23, 13, 23,  1,
         0, 26, 19, 16,  3,  0, 13, 13,  1,  0])


In [4]:
# Splits de treino e validação
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # os primeiros 90% serão treino
train_data = data[:n]
val_data = data[n:]

A ideia vai ser dar pedaços de texto para a rede neural, com o objetivo de prever o caractere seguinte. Para isso, vamos fornecer um certo bloco de texto para a rede: 

In [5]:
block_size = 8
print(train_data[1:block_size+1])

tensor([19, 18, 24, 19,  1,  0,  7, 19])


Mas repare que existem múltiplos exemplos dentro desse bloco. Queremos que a rede saiba prever o próximo caractere quando apenas um caractere de contexto é fornecido, ou quando o bloco inteiro é fornecido.

In [6]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    print(f"Contexto é {x[:t+1]} para prever {y[t]}")

Contexto é tensor([7]) para prever 19
Contexto é tensor([ 7, 19]) para prever 18
Contexto é tensor([ 7, 19, 18]) para prever 24
Contexto é tensor([ 7, 19, 18, 24]) para prever 19
Contexto é tensor([ 7, 19, 18, 24, 19]) para prever 1
Contexto é tensor([ 7, 19, 18, 24, 19,  1]) para prever 0
Contexto é tensor([ 7, 19, 18, 24, 19,  1,  0]) para prever 7
Contexto é tensor([ 7, 19, 18, 24, 19,  1,  0,  7]) para prever 19


In [7]:
torch.manual_seed(0)
batch_size = 4  # Quantas sequências independentes serão processadas em paralelo?
block_size = 8  # Qual é o maior contexto disponível numa sequência?

print(f"Temos {batch_size} exemplos num batch, cada um com tamanho {block_size}\n")

def get_batch(split):
    # Gere um único batch de dados com inputs x e alvos y.
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))  # Um índice aleatório nos dados.
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(' - Input (x):')
print(xb.shape)
print(xb)
print('\n-Alvo (y):')
print(yb.shape)
print(yb)

print('\n--------\nOu seja, isso pode ser entendido como:\n')

for b in range(batch_size): # Dimensão de batch.
    for t in range(block_size): # Dimensão de tempo.
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"Contexto é {context.tolist()} para prever {target}")

Temos 4 exemplos num batch, cada um com tamanho 8

 - Input (x):
torch.Size([4, 8])
tensor([[ 7, 12,  9, 13, 19, 23,  0,  8],
        [ 0, 13, 24, 32, 16, 13,  5,  0],
        [ 5, 26, 13, 16, 12,  5,  3,  0],
        [ 5, 23, 23, 13, 17,  1,  0,  9]])

-Alvo (y):
torch.Size([4, 8])
tensor([[12,  9, 13, 19, 23,  0,  8,  9],
        [13, 24, 32, 16, 13,  5,  0,  8],
        [26, 13, 16, 12,  5,  3,  0,  8],
        [23, 23, 13, 17,  1,  0,  9, 22]])

--------
Ou seja, isso pode ser entendido como:

Contexto é [7] para prever 12
Contexto é [7, 12] para prever 9
Contexto é [7, 12, 9] para prever 13
Contexto é [7, 12, 9, 13] para prever 19
Contexto é [7, 12, 9, 13, 19] para prever 23
Contexto é [7, 12, 9, 13, 19, 23] para prever 0
Contexto é [7, 12, 9, 13, 19, 23, 0] para prever 8
Contexto é [7, 12, 9, 13, 19, 23, 0, 8] para prever 9
Contexto é [0] para prever 13
Contexto é [0, 13] para prever 24
Contexto é [0, 13, 24] para prever 32
Contexto é [0, 13, 24, 32] para prever 16
Contexto é [0,

## 5.2 Criando um modelo bigrama

Vamos começar com um modelo de bigramas, em que dado o caractere atual, queremos prever o seguinte. Ou seja, o contexto tem sempre tamanho 1.

In [8]:
class LanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # Cada linha da token_embedding_table diz os 42 logits do próximo caractere dado o atual.
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # Tanto idx quanto targets são tensores de dimensão (B, T).
        logits = self.token_embedding_table(idx) # (B,T,C)

        return logits

In [9]:
model = LanguageModel(vocab_size)
out = model(xb, yb)
print(out.shape)

torch.Size([4, 8, 42])


Ou seja, para cada um dos 8 caracteres nos 4 exemplos no batch, nós fazemos a previsão de qual é a probabilidade de cada um dos 42 caracteres existentes ser o próximo caractere na sequência. Ou seja, nossas dimensões são:
- batch size `b`: 4 (quantos exemplos temos num batch)
- block size `t`: 8 (quantas previsões em sequência temos em cada exemplo)
- characters `c`: 42 (quantos caracteres podem ser escolhidos na sequência)

E $x_b \in \mathbb{R}^{4 \times 8}$ diz quem é o último caractere em cada exemplo e elemento da sequência. Também, $y_b \in \mathbb{R}^{4 \times 8}$ diz quem é, de fato, o caractere seguinte. Nós estamos usando um embedding que dá a probabilidade de cada um dos $42$ possíveis caracteres serem o caractere seguinte. Logo, nosso `logits` tem dimensão $\mathbb{R}^{4 \times 8 \times 42}$.

Naturalmente, é preciso escolher a matrix `token_embedding_table` de maneira ótima, para que as probabilidades atribuídas a cada caractere façam sentido (e.g., 'xm' é incomum, 'de' é comum). Para isso, vamos adicionar uma perda ao nosso modelo --- em particular, vamos usar a perda logística (ou "entropia cruzada").

In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class LanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # Cada linha da token_embedding_table diz os 42 logits do próximo caractere dado o atual.
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # Tanto idx quanto targets são tensores de dimensão (B, T).
        logits = self.token_embedding_table(idx) # (B,T,C)
        
# ------------------------------------------------------------------------------
        # Para gerar caracteres numa sequência, como na função generate abaixo, não temos targets;
        # nesse caso, também não há perda.
        if targets is None:
            loss = None
        # Para a perda F.cross_entropy, o PyTorch espera que logits tenha C na segunda dimensão.
        # Podemos resolver isso rearranjando os logits em (B*T, C) e os targets de maneira correspondente em (B*T)
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx é um tensor (B, T) que fornece o contexto atual.
        for _ in range(max_new_tokens):
            # Obtenha previsões
            logits, loss = self(idx)
            # Como nosso modelo é um bigrama, dado o contexto, apenas o último caractere dá os logits
            # do possível caractere subsequente.
            logits = logits[:, -1, :]  # (B, C)
            # Obtenha as probabilidades de cada caractere tirando o softmax dos logits.
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Amostre dessas probabildades
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Adicione o índice sorteado ao nosso contexto atual, aumentando a dimensão da sequência.
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
# ------------------------------------------------------------------------------

A função `generate` acima é um pouco dispendiosa. Atualmente, fornecemos todos as sequências que existem dentro de um exemplo (e.g., contexto [0, 26] para prever 13; contexto [0, 26, 13] para prever 9; contexto [0, 26, 13, 9] para prever 22) quando na verdade, por ser um modelo de bigrama, basta o último elemento (e.g., dado [26] prever 13; dado [13] prever 9; dado [9] prever 22). A razão de usar essa função é que, mais abaixo, vamos querer generalizar o modelo de um bigrama para algo que leve em consideração mais contexto. Pela maneira como codificamos `generate`, isso não vai exigir ajustes dessa função.

In [11]:
torch.manual_seed(10)
model = LanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(f"\nPerda: {loss.item()}")

torch.Size([32, 42])

Perda: 4.414548873901367


Repare que também adicionamos a função `generate`, capaz de usar os pesos aprendidos em `token_embedding_table` para sortear caracteres de acordo com os pesos aprendidos. A variável `max_new_tokens` especifica quantos caracteres devem ser gerados.

In [12]:
model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)

tensor([[ 0, 17, 11, 24, 17, 39, 12,  4, 16, 27, 26, 29, 40, 36,  1, 30,  5, 17,
         39,  0, 19, 17,  7, 14, 23, 16,  1, 25, 24, 17, 22, 30, 23, 16, 22, 11,
         26, 38, 33, 27, 12, 20, 35, 34, 34, 35, 37, 18,  1, 10, 24, 17, 32, 39,
         22, 41, 24,  6,  7, 40, 38, 33, 32, 26, 19, 24, 17, 24,  6, 27, 36, 23,
         31, 29,  5,  8, 32, 28, 36,  0,  9,  4, 30, 39, 31,  7, 32, 28, 32,  6,
         13,  6, 39, 31, 26,  0,  7, 32, 31, 29, 18]])

In [13]:
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

 põ, cáíâixvçílíópya,e-úzsnea?átmên,eõíãbxájoxcâwszmaãógvírúàg lad?êc,e-ígm,õúlyez.qíõáàêáld.cççá.?éw


Como não ouve treino, essa amostra é essencialmente aleatória. Na verdade, é pior que isso --- se amostrássemos de maneira uniformemente aleatória, a perda deveria ser

In [14]:
-torch.Tensor([1/vocab_size]).log().item()

3.7376697063446045

Como podemos fazer sentido desse valor? A nossa perda aqui é entropia cruzada , e pode ser entendida como a probabilidade de escolhermos o caractere errado. 

## 5.3 Treinando um modelo bigrama

Vamos treinar o modelo bigrama anterior. Para o otimizador, usaremos Adam, que é uma alternativa um pouco mais sofisticada do que o SGD que usamos antes.

In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

Agora, vamos fazer uma sequência de iterações em que pegamos um batch e damos um passo para otimizar nossa estimativa dos parâmetros.

In [16]:
# torch.manual_seed(10)
# model = LanguageModel(vocab_size)
# logits, loss = model(xb, yb)

batch_size = 32
for steps in range(10000): # use mais passos para obter melhores resultados
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    if steps % 1000 == 0:
        print(f"Iteration {steps}: {loss.item()}")


Iteration 0: 4.173539161682129
Iteration 1000: 3.279672861099243
Iteration 2000: 2.8587005138397217
Iteration 3000: 2.5384347438812256
Iteration 4000: 2.4177489280700684
Iteration 5000: 2.3529484272003174
Iteration 6000: 2.4395017623901367
Iteration 7000: 2.397427797317505
Iteration 8000: 2.3495335578918457
Iteration 9000: 2.256340742111206


In [17]:
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

 o co.ór omamistra neuçandiba ntre e? qussto deesmprdelubíchesqumo, ecirdentemeuvão as ce, ilhao tomo


Ok, isso parece um pouco melhor já.

Antes de continuarmos, vai ser útil organizar um pouco melhor o código.

In [18]:
# Pacotes
import torch
import torch.nn as nn
from torch.nn import functional as F

# Hiperparâmetros
batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

torch.manual_seed(10)

# Dados
with open("../data/gpt/processed/machado-all.txt", "r") as f:
    text = f.read()
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# Splits de treino e validação
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # primeiros 90% serão train, resto val
train_data = data[:n]
val_data = data[n:]

# Função para ler um batch de dados
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# Função para estimar a perda usando vários batches e não um só
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# Nosso modelo bigrama
class LanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = LanguageModel(vocab_size)
model = model.to(device)

# Otimizador
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Treinamento
for iter in range(max_iters):

    # De tempos em tempos, imprimir a perda
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"Iteração {iter}: perda de treino: {losses['train']:.4f}, perda de validação: {losses['val']:.4f}")

    # Amostre um batch de dados
    xb, yb = get_batch('train')

    # Dê um passo na otimização
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Gere alguns caracteres de acordo com o modelo
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print("\n" + decode(model.generate(context, max_new_tokens=500)[0].tolist()))

Iteração 0: perda de treino: 4.1996, perda de validação: 4.1961
Iteração 300: perda de treino: 2.5214, perda de validação: 2.5614
Iteração 600: perda de treino: 2.3496, perda de validação: 2.4021
Iteração 900: perda de treino: 2.3193, perda de validação: 2.3719
Iteração 1200: perda de treino: 2.3211, perda de validação: 2.3707
Iteração 1500: perda de treino: 2.3042, perda de validação: 2.3649
Iteração 1800: perda de treino: 2.2948, perda de validação: 2.3618
Iteração 2100: perda de treino: 2.3012, perda de validação: 2.3556
Iteração 2400: perda de treino: 2.2955, perda de validação: 2.3639
Iteração 2700: perda de treino: 2.3038, perda de validação: 2.3581

 sesa limentar a pradeis amaqunde, qumpfi m doginda nass quisarare mer e nonabeiroça va tum fio asm de m e ptroselha spons. fá do eiz dela a lharitatúnte, rara de é uss, us uerencéitontazum imontyívolapodos pe. toharadasi sulbo mantmospra stuns endenca. o e. canto, toucr aiore u vair de cagupo, vincelempeta qu sto trtes r tia púba. c

Note que acima implementamos duas mudanças técnicas: 
- A função `estimate_loss` que estima a perda usando não apenas um batch, como antes, mas vários, de modo que a estimativa fica mais suave. Note que a função usa `model.eval()` e `model.train()`, o que deixa o modelo em modo de avaliação ou de treinamento; isso não faz diferença agora, porque nossa rede só usa uma camada de `nn.Embedding`, que se comporta da mesma maneira em treino ou avaliação. Como vimos antes, se incluírmos uma camada de batchnorm, esse não será mais o caso.
- O `.to(device)` que coloca o modelo numa GPU, ao invés de CPU. Não vem ao caso entender profundamente porque isso vale a pena além do fato de que, se você tiver uma GPU disponível, o treino provavelmente será muito mais rápido se ele usá-la.

## 5.4 Positional encoding

Vamos começar a criar o decoder. Além do output embedding, que já temos, precisamos de um positional encoding. Como os passos subsequentes vão olhar para o grau de associação entre cada par de caracteres para decidir qual deveria ser o próximo, precisamos ter algum jeito de especificar a sequência que estamos considerando: "era uma grande pedra" e "era uma grande perda" têm os mesmos pares, mas a ordem dos pares muda o sentido do termo e, portanto, do que é razoável de vir a seguir.

Vamos adicionar uma camada `pos_emb` no nosso modelo que codifica a posição de cada par e somá-lo ao nosso embedding dos tokens `tok_emb`. Para deixar que ambos os embeddings assumam uma dimensão arbitrária, e não necessariamente `vocab_size`, vamos introduzir uma última camada linear que leva um tensor de dimensões `(B, T, n_embd)` em `(B, T, vocab_size)`, que é o que precisamos para gerar a probabilidade de cada caractere seguinte.

In [19]:
n_embd = 32

# Nosso modelo bigrama
class LanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)  # Cada posição recebe um embedding
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)  # (B, T, n_embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T, n_embd)
        x = tok_emb + pos_emb  # (B, T, n_embd)
        logits = self.lm_head(x)  # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

Naturalmente, o uso de uma camada de position embedding `pos_embd` não é de muito uso agora, porque, como estamos usando um modelo bigrama, só o último par de caracteres é considerado. Por outro lado, quando começarmos a trabalhar no mecanismo de atenção, isso vai deixar de ser o caso --- e aí poder ter uma maneira de identificar qual é o último e o penúltimo pares, por exemplo, antes do caractere a ser previsto pode ser muito útil.

## 5.5 O mecanismo de auto-atenção

O coração de um transformer, a base do GPT, é o mecanismo de atenção (a célula laranja na primeira figura acima).

In [20]:
torch.manual_seed(0)
B,T,C = 4,8,2 # batch, tempo e caracteres
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

Nosso objetivo é prever quais caracteres são os mais prováveis; queremos fazer isso prevendo o caractere seguinte usando até 8 caracteres de contexto. A ideia vai ser relacionar cada par de caracteres com um grau de associação,  e depois tomar uma média ponderada. 

Mas precisamos tomar um cuidado: para prever, digamos, o sexto caractere, podemos usar até os cinco primeiros caracteres --- mas não o sexto, o sétimo e o oitavo. Ou seja, precisamos de uma máscara, e a maneira mais fácil de fazê-lo é simplesmente dar peso zero, na média ponderada. 

Por ora, vamos simplesmente supor que estamos uma média com peso 1 para cada par.

In [21]:
# Queremos x[b,t] = média_{i<=t} x[b,i], ou seja a média das associações entre todos os tokens que já apareceram
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)  # Média sobre dimensão t, fica com dimensão C

In [22]:
xbow.shape  # (B, T, C)

torch.Size([4, 8, 2])

Essa versão pode ser melhorada: vários dos cálculos podem ser feitos em paralelo. Uma maneira simples do computador se utilizar dessa possibilidade é escrever tudo matricialmente (isso é particularmente verdade quando estamos usando uma GPU). Nesse caso, o único cuidado é dar peso 1 se o caractere já apareceu na sequência, e 0 caso contrário.

In [23]:
torch.tril(torch.ones(T, T))

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [24]:
# Versão 2: usando multiplicação de matrizes
weights = torch.tril(torch.ones(T, T))
weights = weights / weights.sum(1, keepdim=True)
xbow2 = weights @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

Ao invés de normalizar, para que os pesos possam ser interpretados como probabilidade (que é o que fizemos acima), nós podemos simplesmente aplicar um softmax. Nesse caso, basta transformar elements onde `tril == 0` em `-inf`.

In [25]:
# Versão 3: usando softmax
tril = torch.tril(torch.ones(T, T))
weights = torch.zeros((T,T))
weights = weights.masked_fill(tril == 0, float('-inf'))
weights = F.softmax(weights, dim=-1)
xbow3 = weights @ x
torch.allclose(xbow, xbow3)

True

Qual é a vantagem de usar um softmax? É o fato de podemos aprender valores arbitrários para as associações em caracteres em `weights`, inicialmente, e depois deixar `softmax()` transformá-los em probabilidade, como já fazíamos antes.

Como aprender os pesos? Basicamente, queremos que os pesos `weights` sejam aprendidos para refletir afinidades entre pares de caracteres. Por exemplo, se olhamos para uma vogal, provavelmente saber quais consoantes aparecem na mesma frase traz mais informação do que as vogais.

Vamos fazer isso da seguinte maneira: cada letra vai ter associado um valor em duas matrizes
 - `key` $\in \mathbb{R}^{n_{\text{embd}} \times d}$: quem sou eu (e.g., uma vogal). Cada letra (temos `n_embd`) vai ter `d` características.
- `query` $\in \mathbb{R}^{n_{\text{embd}} \times d}$: quais são as informações mais importantes para essa letra querer formar um par (e.g., você é uma consoante?). Cada letra (temos `n_embd`) vai estar interessada em `d` características.

A afinidade de um par, `weights`, vai ser dado pelo produto entre a linha correspondente na matrix `query` e o vetor correspondente na matriz `key`.

In [26]:
# Versão 4: mecanismo de auto-atenção
torch.manual_seed(0)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# Vamos implementar uma Head, isto é, uma maneira de dar atenção a pares
head_size = 16  # Quantas características cada letra pode expressar
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
weights =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# weights = torch.zeros((T,T))  # Antes os pesos eram fixos; agora, vão ser aprendidos
weights = weights.masked_fill(tril == 0, float('-inf'))
weights = F.softmax(weights, dim=-1)

out = weights @ x

out.shape

torch.Size([4, 8, 32])

Quem são os nossos pesos?

In [27]:
weights[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8799, 0.1201, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0562, 0.3655, 0.5783, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1704, 0.4478, 0.0397, 0.3421, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2826, 0.1831, 0.0450, 0.3088, 0.1804, 0.0000, 0.0000, 0.0000],
        [0.4660, 0.1350, 0.2685, 0.0332, 0.0496, 0.0477, 0.0000, 0.0000],
        [0.0463, 0.1805, 0.2533, 0.0089, 0.3825, 0.0815, 0.0468, 0.0000],
        [0.0916, 0.2708, 0.0058, 0.2054, 0.0206, 0.0899, 0.2443, 0.0716]],
       grad_fn=<SelectBackward0>)

Ótimo, de fato a afinidade de um caractere na sequência com qualquer outro é zero toda vez que estamos usando um caractere que ainda não apareceu (e.g., na primeira linha, só observamos o primeiro caractere, logo sua afinidade com todos os outros é zero). Esses pares não podem ter qualquer relação.

Além disso, através desse mecanismo, a rede consegue detectar quais pares são mais úteis para prever o próximo caractere, mesmo que o par tenha ocorrido bem no começo da sequência --- isso era um problema enorme para redes recorrentes. 

Valores altos na matriz acima significa que, ao tomar uma média ponderada, pares com muita afinidade recebem peso maior. A única coisa que falta é dar algum grau de flexibilidade aos valores cuja média ponderada vai ser tomada; `x` em si pode não ser a coisa mais útil --- vamos aprender a transformar cada `x` em um `v` tal que, o output do mecanismo de atenção é uma média ponderada dos valores em `v` (e.g., qual é o valor que um par recebe quando um caractere é vogal e outro é uma consoante?). Uma das vantagens de usar essa matriz `v` é que, se quisermos incluir mais mecanismos de atenção, é possível aprender identidades diferentes para `x` simplesmente mudando essa transformação.

In [28]:
# Versão 4: mecanismo de auto-atenção
torch.manual_seed(0)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# Vamos implementar uma Head, isto é, uma maneira de dar atenção a pares
head_size = 16  # Quantas características cada letra pode expressar
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
weights =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# weights = torch.zeros((T,T))  # Antes os pesos eram fixos; agora, vão ser aprendidos
weights = weights.masked_fill(tril == 0, float('-inf'))
weights = weights * (head_size**-0.5)
weights = F.softmax(weights, dim=-1)

out = weights @ value(x)

Note que a dimensão do output mudou:

In [29]:
out.shape

torch.Size([4, 8, 16])

O que aconteceu?
- `value = x @ v` $\in \mathbb{R}^{B \times T \times C} \times \mathbb{R}^{C \times \text{head\_size}} = \mathbb{R}^{B \times T \times \text{head\_size}}$
- `out = weights @ value` $\in \mathbb{R}^{B \times T \times T} \times \mathbb{R}^{B \times T \times \text{head\_size}} = \mathbb{R}^{B \times T \times \text{head\_size}}$, onde a última desigualdade segue de broadcasting (a múltiplicação é feita para cada elemento em $\mathbb{R}^B$, resultando em $\mathbb{R}^{T \times T} \times \mathbb{R}^{T \times \text{head\_size}}=\mathbb{R}^{T \times T}$).

Ou seja, `x` $\in \mathbb{R}^{B \times T \times C}$, mas `out` $\in \mathbb{R}^{B \times T \times \text{head\_size}}$.

Além disso, dividimos os pesos por $1/\sqrt{\text{head\_size}}$ para que possamos inicializar as matrizes `query` e `key` com variância 1 e manter `weights` na mesma escala. Por quê? Vamos pegar um exemplo e entender o que acontece com a variância de `weights`.

In [30]:
torch.manual_seed(0)

k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
weights = q @ k.transpose(-2, -1)

print(f"Variance of k: {k.var()}")
print(f"Variance of q: {q.var()}")
print(f"Variance of weights: {weights.var()}")

Variance of k: 1.0322861671447754
Variance of q: 1.081316351890564
Variance of weights: 19.013824462890625


Qual é o problema da variância aumentar? Nesse caso, o softmax vai acabar favorecendo um único elemento, e a média ponderada passa a se tornar apenas a escolha de um elemento.

In [31]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [32]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]) * 9, dim=-1)

tensor([0.0228, 0.0015, 0.1382, 0.0015, 0.8359])

Por isso, normalizamos os pesos por $1/\sqrt{\text{head\_size}}$, e garantimos que a rede conseguirá aprender os pesos corretamente:

In [33]:
torch.manual_seed(0)

k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
weights = q @ k.transpose(-2, -1) * head_size**-0.5

print(f"Variance of k: {k.var()}")
print(f"Variance of q: {q.var()}")
print(f"Variance of weights: {weights.var()}")

Variance of k: 1.0322861671447754
Variance of q: 1.081316351890564
Variance of weights: 1.188364028930664


**Positional encoding**:  Note que o mecanismo de atenção é uma maneira de dar um grau de plausibilidade a um próximo caractere, onde essa plausibilidade vem de uma média ponderada da associação do próximo caractere proposto e todos os caracteres que já vieram antes. Isso significa que, a princípio, não estamos favorecendo os caracteres que apareceram mais recentemente, algo essencial em linguagem (depois de um "m" nós não vamos ter um "t", não importa quais outras letras vieram antes). A maneira de resolver isso é colocando o positional encoding, que já estabelecemos preemptivamente. (Repare que a lógica de um mecanismo de atenção é muito diferente de uma convolução, que de fato só dá peso para elementos próximos do elemento sendo considerado.)

**Encoder vs decoder**: Finalmente, o que diferencia um encoder de um decoder no caso de um transformer é o fato de que o decoder só pode se utilizar dos elementos na sequência que já apareceram. Um encoder pode se utilizar de todos os elementos. Do ponto de vista de código, a única diferença é deletar a linha `weights = weights.masked_fill(tril == 0, float('-inf'))`. Na prática, treinar um encoder requer muito mais dados que costumamos ter.

**Outras opções de atenção**: Estamos usando a expressão auto-atenção às vezes, acima. A razão é que os elementos sempre vêm de `x`, isto é, `key(x)`, `query(x)` e `value(x)`. Existem outros mecanismos de atenção, por exemplo em que as queries vêm de x, mas as keys e values vêm do encoder (veja a primeira figura acima, e como um dos blocos de atenção recebe o output do encoder) ou de uma outra fonte externa de informação (isso é chamado de atenção cruzada).

## 5.6 O mecanismo de auto-atenção com multi-Head 


Tendo visto em detalhes o que é um mecanismo de auto-atenção, vamos implementá-lo de maneira organizada ao nosso código usando uma classe.

In [34]:
class Head(nn.Module):
    """Head de um mecanismo de auto-atenção."""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # Precisamos criar tril de uma maneira diferente, dado que seus parâmetros não são treinados
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
    def forward(self, x):
        # Input tem tamanho (batch, time-step, channels)
        # Output tem tamanho (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # Cálculo dos scores de atenção ("afinidades")
        weigths = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        weigths = weigths.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        weigths = F.softmax(weigths, dim=-1) # (B, T, T)
        # Cálculo da média ponderada dos values
        v = self.value(x) # (B,T,hs)
        out = weigths @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


In [35]:
# Nosso modelo bigrama
class LanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)  # Cada posição recebe um embedding
        self.sa_head = Head(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)  # (B, T, n_embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T, n_embd)
        x = tok_emb + pos_emb  # (B, T, n_embd)
        x = self.sa_head(x)  # (B, T, head_size)
        logits = self.lm_head(x)  # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # Por conta do positional embedding, o maior contexto que conseguimos receber é
            # de tamanho block_size; no caso de gerar vários novos caracteres, precisamos
            # restringir o contexto a ter tamanho (máximo) de block_size.
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

Que performance essa rede obtém? Por conta da auto-atenção, vamos reduzir um pouco o learning rate e, correspondentemente, aumentar o número de iterações.

In [36]:
# Hiperparâmetros
batch_size = 32
n_embd = 32
block_size = 8
max_iters = 5000
eval_interval = 300
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

In [37]:
torch.manual_seed(10)

model = LanguageModel(vocab_size)
model = model.to(device)

# Otimizador
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Treinamento
for iter in range(max_iters):

    # De tempos em tempos, imprimir a perda
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"Iteração {iter}: perda de treino: {losses['train']:.4f}, perda de validação: {losses['val']:.4f}")

    # Amostre um batch de dados
    xb, yb = get_batch('train')

    # Dê um passo na otimização
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Gere alguns caracteres de acordo com o modelo
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print("\n" + decode(model.generate(context, max_new_tokens=500)[0].tolist()))

Iteração 0: perda de treino: 3.6880, perda de validação: 3.6867
Iteração 300: perda de treino: 2.6155, perda de validação: 2.6582
Iteração 600: perda de treino: 2.3757, perda de validação: 2.4427
Iteração 900: perda de treino: 2.3186, perda de validação: 2.3772
Iteração 1200: perda de treino: 2.3005, perda de validação: 2.3589
Iteração 1500: perda de treino: 2.2880, perda de validação: 2.3542
Iteração 1800: perda de treino: 2.2634, perda de validação: 2.3477
Iteração 2100: perda de treino: 2.2696, perda de validação: 2.3372
Iteração 2400: perda de treino: 2.2609, perda de validação: 2.3378
Iteração 2700: perda de treino: 2.2565, perda de validação: 2.3241
Iteração 3000: perda de treino: 2.2565, perda de validação: 2.3256
Iteração 3300: perda de treino: 2.2504, perda de validação: 2.3217
Iteração 3600: perda de treino: 2.2394, perda de validação: 2.3194
Iteração 3900: perda de treino: 2.2351, perda de validação: 2.3041
Iteração 4200: perda de treino: 2.2332, perda de validação: 2.3021
I

Nossa perda de validação caiu de 2.36 para 2.30 --- o mecanismo de atenção parece ter ajudado, mesmo que o texto ainda não se pareça com português. Um problema aqui é que cada auto-atenção só pode dar atenção a uma coisa, mas na verdade a decisão de qual deve ser o próximo caractere pode depender de darmos atenção a mais de um fator ao mesmo tempo. Que tal usar mais de uma Head?

In [38]:
class MultiHeadAttention(nn.Module):
    """Mecanismo de auto-atenção com multi-head."""

    def __init__(self, num_heads, head_size):
        super().__init__()
        # Várias Heads são criadas e colocadas numa lista
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self, x):
        # Concatenamos o resultado na última dimensão e retornamos (B, T, sum(head_sizes))
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        return out

In [39]:
# Nosso modelo bigrama
class LanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = MultiHeadAttention(4, n_embd//4)  # 4 Heads com n_embd=8
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)  # (B, T, n_embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T, n_embd)
        x = tok_emb + pos_emb  # (B, T, n_embd)
        x = self.sa_head(x)  # (B, T, head_size)
        logits = self.lm_head(x)  # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # Por conta do positional embedding, o maior contexto que conseguimos receber é
            # de tamanho block_size; no caso de gerar vários novos caracteres, precisamos
            # restringir o contexto a ter tamanho (máximo) de block_size.
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [40]:
torch.manual_seed(10)

model = LanguageModel(vocab_size)
model = model.to(device)

# Otimizador
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Treinamento
for iter in range(max_iters):

    # De tempos em tempos, imprimir a perda
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"Iteração {iter}: perda de treino: {losses['train']:.4f}, perda de validação: {losses['val']:.4f}")

    # Amostre um batch de dados
    xb, yb = get_batch('train')

    # Dê um passo na otimização
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Gere alguns caracteres de acordo com o modelo
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print("\n" + decode(model.generate(context, max_new_tokens=500)[0].tolist()))

Iteração 0: perda de treino: 3.7892, perda de validação: 3.7831
Iteração 300: perda de treino: 2.5108, perda de validação: 2.5548
Iteração 600: perda de treino: 2.3701, perda de validação: 2.4451
Iteração 900: perda de treino: 2.3026, perda de validação: 2.3687
Iteração 1200: perda de treino: 2.2736, perda de validação: 2.3386
Iteração 1500: perda de treino: 2.2558, perda de validação: 2.3233
Iteração 1800: perda de treino: 2.2248, perda de validação: 2.3089
Iteração 2100: perda de treino: 2.2232, perda de validação: 2.2909
Iteração 2400: perda de treino: 2.2063, perda de validação: 2.2891
Iteração 2700: perda de treino: 2.1972, perda de validação: 2.2646
Iteração 3000: perda de treino: 2.1897, perda de validação: 2.2604
Iteração 3300: perda de treino: 2.1793, perda de validação: 2.2552
Iteração 3600: perda de treino: 2.1679, perda de validação: 2.2535
Iteração 3900: perda de treino: 2.1562, perda de validação: 2.2261
Iteração 4200: perda de treino: 2.1492, perda de validação: 2.2240
I

Ótimo, nossa perda caiu um pouco mais --- de 2.30 para 2.22. De fato parece que permitir atenção para múltiplos fatores, mesmo que cada um tenha uma dimensão menor, ajuda --- quem são vogais, quem são letras repetidas, quem já apareceu antes no começo da frase, etc. Estamos progredindo, mesmo que o texto ainda não seja bem o que esperamos do Machado de Assis.

Existe mais uma coisa que podemos fazer aqui, seguindo a proposta do decoder original: adicionar uma camada feedforward totalmente conectada, depois da atenção e antes da camada linear e do softmax. A razão é permitir que, depois de cada caractere observar aquilo que parece merecer sua atenção, essa camada dá possibilidade dos diferentes caracteres cruzarem informações.  

In [41]:
class FeedFoward(nn.Module):
    """Uma camada linear seguida de uma não-linearidade."""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)

In [42]:
# Nosso modelo bigrama
class LanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = MultiHeadAttention(4, n_embd//4)  # 4 Heads com n_embd=8
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.ffwd = FeedFoward(n_embd)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)  # (B, T, n_embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T, n_embd)
        x = tok_emb + pos_emb  # (B, T, n_embd)
        x = self.sa_head(x)  # (B, T, n_embd)
        x = self.ffwd(x) # (B, T, n_embd)
        logits = self.lm_head(x)  # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # Por conta do positional embedding, o maior contexto que conseguimos receber é
            # de tamanho block_size; no caso de gerar vários novos caracteres, precisamos
            # restringir o contexto a ter tamanho (máximo) de block_size.
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [43]:
torch.manual_seed(10)

model = LanguageModel(vocab_size)
model = model.to(device)

# Otimizador
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Treinamento
for iter in range(max_iters):

    # De tempos em tempos, imprimir a perda
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"Iteração {iter}: perda de treino: {losses['train']:.4f}, perda de validação: {losses['val']:.4f}")

    # Amostre um batch de dados
    xb, yb = get_batch('train')

    # Dê um passo na otimização
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

Iteração 0: perda de treino: 3.6751, perda de validação: 3.6748
Iteração 300: perda de treino: 2.4648, perda de validação: 2.5100
Iteração 600: perda de treino: 2.3232, perda de validação: 2.4006
Iteração 900: perda de treino: 2.2712, perda de validação: 2.3353
Iteração 1200: perda de treino: 2.2401, perda de validação: 2.3089
Iteração 1500: perda de treino: 2.2196, perda de validação: 2.2910
Iteração 1800: perda de treino: 2.1896, perda de validação: 2.2719
Iteração 2100: perda de treino: 2.1896, perda de validação: 2.2603
Iteração 2400: perda de treino: 2.1708, perda de validação: 2.2436
Iteração 2700: perda de treino: 2.1603, perda de validação: 2.2374
Iteração 3000: perda de treino: 2.1638, perda de validação: 2.2397
Iteração 3300: perda de treino: 2.1489, perda de validação: 2.2275
Iteração 3600: perda de treino: 2.1397, perda de validação: 2.2279
Iteração 3900: perda de treino: 2.1324, perda de validação: 2.2156
Iteração 4200: perda de treino: 2.1241, perda de validação: 2.2005
I

Mais uma melhora: de 2.22 para 2.20. No total, avançamos até agora de 2.36 para 2.20. 

Além disso, nossa rede agora tem uma estrutura interessante: o bloco de atenção comunica a relevância de cada par de caracteres, e depois a camada feedforward reúne as informações e calcula o que fazer. Vamos concretizar esse entendimento criando um bloco reunindo essa estrutura.

In [44]:
class Block(nn.Module):
    """Block transformer: primeiro, comunicação; depois, cálculo"""

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)

    def forward(self, x):
        x = self.sa(x)
        x = self.ffwd(x)
        return x
    
    
class LanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
        )
#         self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

Agora, a rede está ficando muito grande, com muitos parâmetros e camadas. Vamos nos valer de três ideias para evitar que isso seja um problema: (i) conexões residuais; (ii) normalização; (iii) dropout.

## 5.7 Conexões residuais

Um problema em ter uma rede muito grande é que ficamos muito propensos a ter gradientes explosivos ou dissipados. Isto é, como a regra da cadeia envolve multiplicar termos, acabamos multiplicando vários termos grandes (e o gradiente vai pra infinito) ou pequenos (e o gradiente vai pra zero). No caso de uma ativação como ReLU, aproximadamente metade das vezes o gradiente é exatamente zero, o que eliminaria o aprendizado para alguns pesos.

Uma ideia simples para evitar que gradientes se dissipem é criar um caminho direto entre o input e o output (com gradiente um), e adicionar as camadas como um caminho alternativo (no começo as camadas são inicializadas aleatoriamente, portanto contribuem pouco para o aprendizado e têm um gradiente pequeno). Aos poucos o aprendizado começa a acontecer, e a importância desses caminhos alternativos vai aumentando, o que permite que os gradientes se alterem de maneira estável.

![](https://i.stack.imgur.com/AdBoF.png)

(Uma outra vantagem é que, assim como o positional encoding, o fato do input ser passado adiante até a última camada significa que a ordem dos caracteres também fica disponível em todos os níveis da rede.)

In [45]:
class Block(nn.Module):
    """Block transformer: primeiro, comunicação; depois, cálculo"""

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)

# ---------------------------------------------------------------------------------------------------
    def forward(self, x):
#         x = self.sa(x)
        x = x + self.sa(x)
#         x = self.ffwd(x)
        x = x + self.ffwd(x)
        return x
# ---------------------------------------------------------------------------------------------------    

Um problema ao fazer isso é que `x` e a saída de `MultiHeadAttention` e `FeedFoward` precisam ter as mesmas dimensões, para garantir que as operações `x + self.sa(x)` e `x + self.ffwd(x)` sejam válidas. Uma maneira de sempre garantir isso é adicionando uma camada de projeção em cada camada (no nosso caso, ela não é necessária, mas dá mais expressividade ao modelo).

In [46]:
class MultiHeadAttention(nn.Module):
    """Mecanismo de auto-atenção com multi-head."""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
# ---------------------------------------------------------------------------------------------------    
        self.proj = nn.Linear(head_size * num_heads, n_embd)
# ---------------------------------------------------------------------------------------------------    

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
# ---------------------------------------------------------------------------------------------------    
        out = self.proj(out) 
# ---------------------------------------------------------------------------------------------------    
        return out

class FeedFoward(nn.Module):
    """Uma camada linear seguida de uma não-linearidade."""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),  # Para seguir o artigo original, dando mais liberdade.
            nn.ReLU(),
# ---------------------------------------------------------------------------------------------------    
            nn.Linear(4 * n_embd, n_embd),
# ---------------------------------------------------------------------------------------------------    
        )

    def forward(self, x):
        return self.net(x)

Agora, podemos treinar novamente nosso modelo.

In [47]:
class LanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
        )
#         self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # Por conta do positional embedding, o maior contexto que conseguimos receber é
            # de tamanho block_size; no caso de gerar vários novos caracteres, precisamos
            # restringir o contexto a ter tamanho (máximo) de block_size.
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [48]:
n_embd = 32

torch.manual_seed(10)

model = LanguageModel()
model = model.to(device)

# Otimizador
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Treinamento
for iter in range(max_iters):

    # De tempos em tempos, imprimir a perda
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"Iteração {iter}: perda de treino: {losses['train']:.4f}, perda de validação: {losses['val']:.4f}")

    # Amostre um batch de dados
    xb, yb = get_batch('train')

    # Dê um passo na otimização
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

Iteração 0: perda de treino: 4.1696, perda de validação: 4.1782
Iteração 300: perda de treino: 2.3090, perda de validação: 2.3771
Iteração 600: perda de treino: 2.2049, perda de validação: 2.2838
Iteração 900: perda de treino: 2.1622, perda de validação: 2.2567
Iteração 1200: perda de treino: 2.1195, perda de validação: 2.2166
Iteração 1500: perda de treino: 2.0886, perda de validação: 2.1573
Iteração 1800: perda de treino: 2.0674, perda de validação: 2.1472
Iteração 2100: perda de treino: 2.0489, perda de validação: 2.1291
Iteração 2400: perda de treino: 2.0346, perda de validação: 2.1211
Iteração 2700: perda de treino: 2.0257, perda de validação: 2.0985
Iteração 3000: perda de treino: 2.0055, perda de validação: 2.0932
Iteração 3300: perda de treino: 2.0004, perda de validação: 2.0896
Iteração 3600: perda de treino: 1.9929, perda de validação: 2.0795
Iteração 3900: perda de treino: 1.9774, perda de validação: 2.0750
Iteração 4200: perda de treino: 1.9815, perda de validação: 2.0724
I

Fantástico! Fomos de 2.20 para 2.05. Será que agora já estamos perto de português?

In [49]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print("\n" + decode(model.generate(context, max_new_tokens=500)[0].tolist()))


 mãos., ecoisão da manos é de do a graia dem mordado o dentre minho não antido, mas -lhe com comenterias lesta mesmo que pulio, não genfoite, é cogoração periveiasses era um contas do poco ele predifissõos. fesfar espais jaé nos manhoce. ção é um ele desto re deixa em adesse poi há do dele que tansias do coit, mamentos tam sel, expo da soitor o cooecre arclenes há que fais tenha implete contão. da chora era era exprovoro. de reis. veze combrias que são sangado. pada os vanidades das da zorde? o i


Hmm, é uma melhora substancial. Ainda faltam uns truques para reduzir ainda mais essa perda.

## 5.8 LayerNorm

Já vimos a ideia de batchnorm e o impacto que essa camada pode ter no treinamento, em particular suavizando os gradientes. No caso de um GPT, os autores utilizaram uma layernorm, que é similar: ao invés de garantir média zero e variância um nos pontos dentro de um batch, vamos fazê-lo dentro da camada. É uma mudança trivial, do ponto de vista do código que tínhamos para `BatchNorm1d`:

In [50]:
# class BatchNorm1d:
class LayerNorm1d:
  
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
  
    def __call__(self, x):
# ---------------------------------------------------------------------------------------------------    
#         xmean = x.mean(0, keepdim=True) # média do batch
        xmean = x.mean(1, keepdim=True) # média da camada
#         xvar = x.var(0, keepdim=True) # variância do batch
        xvar = x.var(1, keepdim=True) # variância da camada
# ---------------------------------------------------------------------------------------------------    
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out
  
    def parameters(self):
        return [self.gamma, self.beta]

In [51]:
torch.manual_seed(0)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

Ou seja, o batch tem dimensão 32, e para cada elemento do batch as features têm dimensão 100.

In [52]:
x[:,0].mean(), x[:,0].std() # média, desvio-padrão de uma feature ao longo de todos os 32 elementos do batch

(tensor(-0.3143), tensor(1.2249))

In [53]:
x[0,:].mean(), x[0,:].std() # média, desvio-padrão de um elemento do batch ao longo de todas as 100 features

(tensor(-1.1921e-08), tensor(1.0000))

(Note que, acima ignormamos as diferenças em modo de treino e de teste: isso era um problema para batchnorm porque o tamanho do batch poderia mudar; o tamanho da camada é sempre o mesmo.)

Agora, vamos implementar a rede com layernorm. Uma mudança, em relação à figura inicial acima, é que vamos aplicar a normalização antes da transformação linear, e não depois. Com o tempo, isso se mostrou uma alternativa melhor.

In [54]:
class Block(nn.Module):
    """Block transformer: primeiro, comunicação; depois, cálculo"""

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
# ---------------------------------------------------------------------------------------------------    
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
# ---------------------------------------------------------------------------------------------------    


    def forward(self, x):
# ---------------------------------------------------------------------------------------------------    
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
# ---------------------------------------------------------------------------------------------------    
        return x

In [55]:
class LanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            nn.LayerNorm(n_embd)
        )
# ---------------------------------------------------------------------------------------------------    
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
# ---------------------------------------------------------------------------------------------------    
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
# ---------------------------------------------------------------------------------------------------
        x = self.ln_f(x)
# ---------------------------------------------------------------------------------------------------
        logits = self.lm_head(x) # (B,T,vocab_size)    

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # Por conta do positional embedding, o maior contexto que conseguimos receber é
            # de tamanho block_size; no caso de gerar vários novos caracteres, precisamos
            # restringir o contexto a ter tamanho (máximo) de block_size.
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [56]:
n_embd = 32

torch.manual_seed(10)

model = LanguageModel()
model = model.to(device)

# Otimizador
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Treinamento
for iter in range(max_iters):

    # De tempos em tempos, imprimir a perda
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"Iteração {iter}: perda de treino: {losses['train']:.4f}, perda de validação: {losses['val']:.4f}")

    # Amostre um batch de dados
    xb, yb = get_batch('train')

    # Dê um passo na otimização
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

Iteração 0: perda de treino: 3.9061, perda de validação: 3.9092
Iteração 300: perda de treino: 2.3166, perda de validação: 2.3758
Iteração 600: perda de treino: 2.2150, perda de validação: 2.2881
Iteração 900: perda de treino: 2.1563, perda de validação: 2.2468
Iteração 1200: perda de treino: 2.1244, perda de validação: 2.2144
Iteração 1500: perda de treino: 2.0950, perda de validação: 2.1654
Iteração 1800: perda de treino: 2.0698, perda de validação: 2.1511
Iteração 2100: perda de treino: 2.0575, perda de validação: 2.1429
Iteração 2400: perda de treino: 2.0385, perda de validação: 2.1265
Iteração 2700: perda de treino: 2.0332, perda de validação: 2.1049
Iteração 3000: perda de treino: 2.0075, perda de validação: 2.0978
Iteração 3300: perda de treino: 2.0072, perda de validação: 2.0963
Iteração 3600: perda de treino: 1.9885, perda de validação: 2.0729
Iteração 3900: perda de treino: 1.9744, perda de validação: 2.0694
Iteração 4200: perda de treino: 1.9756, perda de validação: 2.0674
I

Hmm, parece que a layernorm não nos ajudou tanto: fomos de 2.0517 para 2.0433. Mas, por outro lado, repare que estamos com a perda de treino muito abaixo da de validação. Talvez devéssemos nos preocupar em reduzir o overfitting. Uma solução é o dropout --- vamos adicioná-lo em algumas camadas. Na mesma medida em que vamos nos preocupar em prevenir overfitting, podemos aumentar a escala do modelo.

## 5.8 Modelo final

In [57]:
batch_size = 64
block_size = 256 # tamanho do contexto agora é muito maior!
max_iters = 15000
eval_interval = 500
learning_rate = 3e-4 # 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 8
n_layer = 8
dropout = 0.2

torch.manual_seed(0);

In [58]:
class Head(nn.Module):
    """Head de um mecanismo de auto-atenção."""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
# ---------------------------------------------------------------------------------------------------
        self.dropout = nn.Dropout(dropout)
# ---------------------------------------------------------------------------------------------------

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        weights = q @ k.transpose(-2,-1) * C**-0.5
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        weights = F.softmax(weights, dim=-1)
# ---------------------------------------------------------------------------------------------------
        weights = self.dropout(weights)  # Algumas conexões aleatoriamente deixam de ser usadas por robustez
# ---------------------------------------------------------------------------------------------------
        v = self.value(x)
        out = weights @ v
        return out

class MultiHeadAttention(nn.Module):
    """Mecanismo de auto-atenção com multi-head."""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
# ---------------------------------------------------------------------------------------------------
        self.dropout = nn.Dropout(dropout)
# ---------------------------------------------------------------------------------------------------

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
# ---------------------------------------------------------------------------------------------------
        out = self.dropout(self.proj(out))
# ---------------------------------------------------------------------------------------------------
        return out

class FeedFoward(nn.Module):
    """Uma camada linear seguida de uma não-linearidade."""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
# ---------------------------------------------------------------------------------------------------
            nn.Dropout(dropout),
# ---------------------------------------------------------------------------------------------------
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """Block transformer: primeiro, comunicação; depois, cálculo"""

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


In [59]:
class GPTDecoderModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
# ---------------------------------------------------------------------------------------------------
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
# ---------------------------------------------------------------------------------------------------
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [60]:
model = GPTDecoderModel()
m = model.to(device)
print(f"Número de parâmetros: {sum(p.numel() for p in m.parameters())/1e6}M")

Número de parâmetros: 14.317866M


In [61]:
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# for iter in range(max_iters):

#     if iter % eval_interval == 0 or iter == max_iters - 1:
#         losses = estimate_loss()
#         print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

#     xb, yb = get_batch('train')

#     logits, loss = model(xb, yb)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step()

In [62]:
# step 0: train loss 3.8188, val loss 3.8246
# step 500: train loss 2.1604, val loss 2.2320
# step 1000: train loss 1.7055, val loss 1.8254
# step 1500: train loss 1.5085, val loss 1.6635
# step 2000: train loss 1.4085, val loss 1.5826
# step 2500: train loss 1.3511, val loss 1.5290
# step 3000: train loss 1.3079, val loss 1.4974
# step 3500: train loss 1.2755, val loss 1.4709
# step 4000: train loss 1.2579, val loss 1.4581
# step 4500: train loss 1.2368, val loss 1.4317
# step 5000: train loss 1.2192, val loss 1.4123
# step 5500: train loss 1.2053, val loss 1.4128
# step 6000: train loss 1.1920, val loss 1.3997
# step 6500: train loss 1.1815, val loss 1.3830
# step 7000: train loss 1.1688, val loss 1.3781
# step 7500: train loss 1.1600, val loss 1.3645
# step 8000: train loss 1.1539, val loss 1.3610
# step 8500: train loss 1.1440, val loss 1.3552
# step 9000: train loss 1.1324, val loss 1.3372
# step 9500: train loss 1.1286, val loss 1.3373
# step 10000: train loss 1.1215, val loss 1.3298
# step 10500: train loss 1.1136, val loss 1.3237
# step 11000: train loss 1.1096, val loss 1.3214
# step 11500: train loss 1.1039, val loss 1.3160
# step 12000: train loss 1.1019, val loss 1.3158
# step 12500: train loss 1.0944, val loss 1.3139
# step 13000: train loss 1.0922, val loss 1.3121
# step 13500: train loss 1.0886, val loss 1.3101
# step 14000: train loss 1.0815, val loss 1.3078
# step 14500: train loss 1.0799, val loss 1.3054
# step 14999: train loss 1.0754, val loss 1.3058

Antes, com redes neurais de muitas camadas, estávamos em ~1.7. Em algumas iterações, já estamos em 1.3! Por outro lado, a rede é muito mais lenta, e repare que a escala que adicionamos é fundamental: antes, o melhor que conseguimos com nossa rede GPTDecoderModel foi 2.04!

Será que agora, finalmente, temos algo que parece Machado de Assis?

In [63]:
# torch.manual_seed(10);
# context = torch.zeros((1, 1), dtype=torch.long, device=device)
# print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

_uma série de objetos que precisasse da música de madri e da situação não de vida oficial. só isso? acho um súdito relíquido isto não teve pressa. seu óculo, falo do que eu vi casar-se ama-me, e não menos do que eu quincassibasse tanto ou fazia calar o tempo depois. seste ano para sábio, e os últimos primeiros tantos anos, quem ali nho mundo, onde ama? olhe, quem tem saídas? ó filha. ia a cortesia do primeiro, queres, posto quem visse uma página de terra, pouca vez tentava passo de amar para outr_